# Korean LLM (Large Language Model) fine-tuning on SageMaker
---

- 허깅페이스 인증 정보 설정: `huggingface-cli login`
    - https://huggingface.co/join
    - https://huggingface.co/settings/tokens
    

## Overview 
바로 이전 모듈까지는 기존에 온프레미스에서 개발했던 환경과 동일한 환경으로 모델을 빌드하고 훈련했습니다. 하지만 아래와 같은 상황들에서도 기존 환경을 사용하는 것이 바람직할까요?

- 온프레미스의 GPU가 총 1장으로 훈련 시간이 너무 오래 소요됨
- 가용 서버 대수가 2대인데 10개의 딥러닝 모델을 동시에 훈련해야 함
- 필요한 상황에만 GPU를 활용

Amazon SageMaker는 데이터 과학자들 및 머신 러닝 엔지니어들을 위한 완전 관리형 머신 러닝 서비스로 훈련 및 추론 수행 시 인프라 설정에 대한 추가 작업이 필요하지 있기에, 단일 GPU 기반의 딥러닝 훈련을 포함한 멀티 GPU 및 멀티 인스턴스 분산 훈련을 보다 쉽고 빠르게 수행할 수 있습니다. SageMaker는 다양한 유즈케이스들에 적합한 예제들을 지속적으로 업데이트하고 있으며, 한국어 세션 및 자료들도 제공되고 있습니다.

### Note
- 이미 기본적인 Hugging Face 용법 및 자연어 처리에 익숙하신 분들은 앞 모듈을 생략하고 이 모듈부터 핸즈온을 시작하셔도 됩니다.
- 이 노트북은 SageMaker 기본 API를 참조하므로, SageMaker Studio, SageMaker 노트북 인스턴스 또는 AWS CLI가 설정된 로컬 시스템에서 실행해야 합니다. SageMaker Studio 또는 SageMaker 노트북 인스턴스를 사용하는 경우 PyTorch 기반 커널을 선택하세요.
- 훈련(Training) job 수행 시 최소 `ml.g5.2xlarge` 훈련 인스턴스를 권장하며, 분산 훈련 수행 시에는 `ml.g5.12xlarge` 훈련 인스턴스를 권장합니다. 만약 인스턴스 사용에 제한이 걸려 있다면 [Request a service quota increase for SageMaker resources](https://docs.aws.amazon.com/sagemaker/latest/dg/regions-quotas.html#service-limit-increase-request-procedure)를 참조하여 인스턴스 제한을 해제해 주세요.


In [75]:
%store -r bucket_prefix dataset_prefix_50_samples s3_data_path s3_data_path_50_samples s3_data_path_10000_samples

In [23]:
try:
    bucket_prefix
    dataset_prefix
    s3_data_path
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] 1번 모듈 노트북을 다시 실행해 주세요.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

++++++++++++++++++++++++++++++++++++++++++++++++++++++++
[ERROR] 1번 모듈 노트북을 다시 실행해 주세요.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++


<br>

## 1. Download LLM from Hugging Face hub
---

In [24]:
!mkdir -p /home/ec2-user/SageMaker/models

In [25]:
import os
from pathlib import Path
from huggingface_hub import snapshot_download

HF_MODEL_ID = "nlpai-lab/kullm-polyglot-12.8b-v2"

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

# create model dir
model_name = HF_MODEL_ID.split("/")[-1].replace('.', '-')
model_tar_dir = Path(f"/home/ec2-user/SageMaker/models/{model_name}")
if not os.path.isdir(model_tar_dir):
    os.makedirs(model_tar_dir, exist_ok=True)
    # Download model from Hugging Face into model_dir
    snapshot_download(
        HF_MODEL_ID, 
        local_dir=str(model_tar_dir), 
        local_dir_use_symlinks=False,
        allow_patterns=allow_patterns,
        cache_dir="/home/ec2-user/SageMaker/"
    )

<br>

## 2. Save LLM to S3
---


In [26]:
import sagemaker
import boto3
sess = sagemaker.Session()
region = boto3.Session().region_name
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
bucket = None
if bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=bucket)

print(f"SageMaker role arn: {role}")
print(f"SageMaker bucket: {sess.default_bucket()}")
print(f"SageMaker session region: {sess.boto_region_name}")

SageMaker role arn: arn:aws:iam::057716757052:role/dt2gsmoon
SageMaker bucket: sagemaker-us-east-1-057716757052
SageMaker session region: us-east-1


In [27]:
s3_pretrained_model_path = f"s3://{bucket}/{bucket_prefix}/huggingface-models/{model_name}/"
s3_chkpt_path= f"s3://{bucket}/{bucket_prefix}/huggingface-models/{model_name}/checkpoints"

In [28]:
%%bash
aws configure set default.s3.max_concurrent_requests 100
aws configure set default.s3.max_queue_size 10000
aws configure set default.s3.multipart_threshold 1GB
aws configure set default.s3.multipart_chunksize 64MB

In [29]:
!aws s3 sync {model_tar_dir} {s3_pretrained_model_path}

<br>

## 3. SageMaker Training
---

SageMaker에 대한 대표적인 오해가 여전히 많은 분들이 SageMaker 훈련을 위해 소스 코드를 전면적으로 수정해야 한다고 생각합니다. 하지만, 실제로는 별도의 소스 코드 수정 없이 기존 여러분이 사용했던 파이썬 스크립트에 SageMaker 훈련에 필요한 SageMaker 전용 환경 변수들만 추가하면 됩니다.

SageMaker 훈련은 훈련 작업을 호출할 때, 1) 훈련 EC2 인스턴스 프로비저닝 - 2) 컨테이너 구동을 위한 도커 이미지 및 훈련 데이터 다운로드 - 3) 컨테이너 구동 - 4) 컨테이너 환경에서 훈련 수행 - 5) 컨테이너 환경에서 S3의 특정 버킷에 저장 - 6) 훈련 인스턴스 종료로 구성됩니다. 따라서, 훈련 수행 로직은 아래 예시와 같이 기존 개발 환경과 동일합니다.

`/opt/conda/bin/python train_hf.py --num_epochs 5 --train_batch_size 32 ...`

이 과정에서 컨테이너 환경에 필요한 환경 변수(예: 모델 경로, 훈련 데이터 경로) 들은 사전에 지정되어 있으며, 이 환경 변수들이 설정되어 있어야 훈련에 필요한 파일들의 경로를 인식할 수 있습니다. 대표적인 환경 변수들에 대한 자세한 내용은 https://github.com/aws/sagemaker-containers#important-environment-variables 을 참조하세요.

### Setup SageMaker Estimator

In [30]:
USE_WANDB = False
LOCAL_MODE = False
# LOCAL_MODE = True

if USE_WANDB:
    import wandb
    wandb.login()

In [31]:
if USE_WANDB:
    wandb.sagemaker_auth(path="src")
    entry_point = "run-wandb.sh"
else:
    entry_point = "run.sh"

In [82]:
instance_type = 'local_gpu' if LOCAL_MODE else 'ml.g5.48xlarge'
print(instance_type)

if instance_type in ['local', 'local_gpu']:
    from sagemaker.local import LocalSession
    sm_session = LocalSession()
    sm_session.config = {'local': {'local_code': True}}
else:
    sm_session = sagemaker.session.Session()

ml.g5.48xlarge


#### SageMaker Training
- Base Container image link : https://github.com/aws/deep-learning-containers/blob/master/available_images.md
- CloudWatch 에 사용자 정의 Metrics 를 추가 참조 (예: Training Loss)
    - [Monitor and Analyze Training Jobs Using Amazon CloudWatch Metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/training-metrics.html)

In [85]:
import time
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.inputs import TrainingInput
from sagemaker.pytorch import PyTorch
import boto3
import sagemaker

# Define Training Job Name 
job_name = name_from_base(f"{model_name}-lora-peft")

# See https://github.com/aws/deep-learning-containers/blob/master/available_images.md
image_uri = f'763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training:2.1.0-gpu-py310-cu121-ubuntu20.04-sagemaker'
hparams = {}

# max_run = 6*60*60 # 6 hours
max_run = 72*60*60 # 3 days

use_spot_instances = False
if use_spot_instances:
    max_wait = 12*60*60 # 12 hours: spot instance waiting + max runtime
else:
    max_wait = None

# Emit: {'loss': 1.0877, 'learning_rate': 6.860813704496788e-05, 'epoch': 0.78} captured by CloudWatch

metric_definitions=[
    {"Name": "train:loss", "Regex": "'loss':(.*?),"}
]

# Create the Estimator
estimator = PyTorch(
    image_uri=image_uri,
    entry_point=entry_point,        # train script
    source_dir='../src',               # directory which includes all the files needed for training
    instance_type=instance_type,    # instances type used for the training job
    instance_count=1,               # the number of instances used for training
    base_job_name=job_name,         # the name of the training job
    role=role,                      # Iam role used in training job to access AWS ressources, e.g. S3
    sagemaker_session=sm_session,   # sagemaker session
    volume_size=300,                # the size of the EBS volume in GB
    hyperparameters=hparams,
    debugger_hook_config=False,
    disable_profile=True,
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait if use_spot_instances else None,
    checkpoint_s3_uri=s3_chkpt_path if instance_type not in ['local', 'local_gpu'] else None,
    checkpoint_local_path='/opt/ml/checkpoints' if instance_type not in ['local', 'local_gpu'] else None,
    metric_definitions = metric_definitions
    #environment={"TRANSFORMERS_OFFLINE": "1", "HF_DATASETS_OFFLINE":"1"},
)

### Start Training job
S3에서 훈련 인스턴스로 복사될 데이터를 지정한 후 SageMaker 훈련 job을 시작합니다. 모델 크기, 데이터 세트 크기에 따라서 몇십 분에서 몇 시간까지 소요될 수 있습니다.
- [SageMaker Experiment Doc](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-experiments/sagemaker_job_tracking/pytorch_script_mode_training_job.html)

In [86]:
if LOCAL_MODE:
    estimator.fit(
        {
            "pretrained": f'file://../../../../../../SageMaker/models/{model_name}', # VS Code on SageMaker Notebook Instance
            # "pretrained": f'file://../../models/{model_name}', # Jupyter Notebook on SageMaker Notebook Instance
            "training": f'file://../{dataset_prefix}'
        },
        wait=False
    )
else:
    from sagemaker.experiments.run import Run
    from sagemaker.utils import unique_name_from_base
    from sagemaker.session import Session

    # set new experiment configuration
    experiment_name = f"{HF_MODEL_ID.split('/')[1].split('.')[0]}"
    # experiment_name = unique_name_from_base(experiment_name)
    
    run_name = f"training-job-experiment"
    print(f"experiment_name:{experiment_name}")    

    with Run(experiment_name=experiment_name, run_name=run_name, sagemaker_session=Session()) as run:
        fast_file = lambda x: TrainingInput(x, input_mode="FastFile")
        estimator.fit(
            {
                "pretrained": fast_file(s3_pretrained_model_path),
                #"training": fast_file(s3_data_path_50_samples),
                # "training": fast_file(s3_data_path_10000_samples),                                
                "training": fast_file(s3_data_path),
            },
            wait=False
        )

    from IPython.display import display, HTML

    def make_console_link(region, train_job_name, train_task='[Training]'):
        train_job_link = f'<b> {train_task} Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/jobs/{train_job_name}">Training Job</a></b>'   
        cloudwatch_link = f'<b> {train_task} Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={region}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={train_job_name};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a></b>'
        return train_job_link, cloudwatch_link  

    train_job_name = estimator.latest_training_job.job_name
    train_job_link, cloudwatch_link = make_console_link(region, train_job_name, '[Fine-tuning]')

    display(HTML(train_job_link))
    display(HTML(cloudwatch_link))

experiment_name:kullm-polyglot-12


INFO:sagemaker.experiments.run:The run (training-job-experiment) under experiment (kullm-polyglot-12) already exists. Loading it.
INFO:sagemaker:Creating training-job with name: kullm-polyglot-12-8b-v2-lora-peft-2024--2024-04-07-11-16-41-377


### View Logs
훈련 로그는 CloudWatch Logs를 통해서 확인할 수 있습니다. 만약 다른 코드 셀을 실행하고 싶다면 이 코드 셀의 실행을 중단하셔도 됩니다.

In [60]:
estimator.logs()

2024-04-07 10:07:08 Starting - Starting the training job
2024-04-07 10:07:08 Pending - Training job waiting for capacity......
2024-04-07 10:08:02 Pending - Preparing the instances for training......
2024-04-07 10:08:57 Downloading - Downloading input data...
2024-04-07 10:09:33 Downloading - Downloading the training image..................
2024-04-07 10:12:30 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-04-07 10:12:31,376 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-04-07 10:12:31,430 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-04-07 10:12:31,440 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-04-07 10:12:31,441 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-04-07 10:12

KeyboardInterrupt: 

training-job-experiment-1712469870-497e


### (Optional) Copy S3 model artifact to local directory
S3에 저장된 모델 아티팩트를 로컬 경로로 복사하여 압축을 해제합니다. 필요 시 로컬 환경에서 모델을 로드하여 추론을 수행할 수 있습니다.

In [ ]:
import json, os

local_model_dir = 'model_from_sagemaker'

if not os.path.exists(local_model_dir):
    os.makedirs(local_model_dir)

!aws s3 cp {estimator.model_data} {local_model_dir}/model.tar.gz
!tar -xzf {local_model_dir}/model.tar.gz -C {local_model_dir}
!rm {local_model_dir}/model.tar.gz

In [ ]:
%store train_job_name

In [87]:
! pip list | grep torch


torch                     2.1.0
torch-model-archiver      0.7.1b20230208
torch-workflow-archiver   0.2.11b20231012
torchaudio                2.1.0
torchdata                 0.7.0
torchserve                0.8.2b20230828
torchtext                 0.16.0
torchvision               0.16.0
